In [1]:
cd .. 

/Users/HPhuc/Practice/8. Kits2019/segment3D


In [2]:
# !pip install -r requirements.txt

In [3]:
import pandas as pd 
import numpy as np
import torch
import torch.nn as nn 

/Users/HPhuc/miniconda3/envs/venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from segment.models.unet3D import BasicUnet3D, UnetRes3D_v1, UnetRes3D_v2, DyUnetRes3D
from segment.training.trainer.trainer import Trainer
from segment.training.losses.loss import Dice_BCE, FocalDiceLoss, DiceLoss
from segment.training.metrics.metric import Dice_3D

In [5]:
data = pd.read_csv('data/train_ds.csv', index_col=0)
data 

,case_id,new_vol_path,new_seg_path
0,case_00207,/Users/HPhuc/Practice/8. Kits2019/Source/outpu...,/Users/HPhuc/Practice/8. Kits2019/Source/outpu...
1,case_00190,/Users/HPhuc/Practice/8. Kits2019/Source/outpu...,/Users/HPhuc/Practice/8. Kits2019/Source/outpu...
2,case_00151,/Users/HPhuc/Practice/8. Kits2019/Source/outpu...,/Users/HPhuc/Practice/8. Kits2019/Source/outpu...
3,case_00129,/Users/HPhuc/Practice/8. Kits2019/Source/outpu...,/Users/HPhuc/Practice/8. Kits2019/Source/outpu...


In [6]:
# pretrained_path = '/content/drive/MyDrive/Seg3D/KiTS2019/Source/outputs/checkpoints/best_UnetRes3D_softmax_fold0_20220509.pt'
pretrained_path = '/content/drive/MyDrive/Seg3D/KiTS2019/Source/outputs/checkpoints/best_UnetRes3D_softmax_fold0_20220508_train.pt'

def segment(pretrained_path, freeze_feature, num_classes=3, act='sigmoid'):

  # backbone = DyUnetRes3D('resnet50', num_classes=num_classes)
  # backbone = UnetRes3D_v1(in_channels=1, num_classes)
  backbone = UnetRes3D_v2(in_channels=1, num_classes=num_classes)
  
  if freeze_feature:
    print('freeze_feature')
    child = nn.Sequential(*list(backbone.children())[:-1])
    for param in child.parameters():
      param.requires_grad = False  

  model = nn.Sequential()
  model.backbone = backbone

  if act == 'sigmoid':
    model.act = nn.Sigmoid()
  elif act == 'softmax':
    model.act = nn.Softmax(dim=1)

  if pretrained_path is not None:
    cp = torch.load(pretrained_path)
    model.load_state_dict(cp['model_state_dict'])
    print(pretrained_path)
    print(sum(p.numel() for p in model.parameters()))

  return model

model = segment(pretrained_path=None, freeze_feature=False, act='softmax')


In [7]:
# x = torch.rand(1,2,80,160,160)
# print(model(x).shape)
# print(sum(p.numel() for p in model.parameters()))

In [8]:
import torch.optim as optim
import segment.utils.parameter as para 

class_weight = np.array([0.00673965, 0.27936378, 1.0])
# class_weight = np.array([0.27936378, 1.0])
class_weight = class_weight.reshape(1, para.num_classes, *[1] * 3)

opt_configs = {
    'Adam': torch.optim.Adam(model.parameters(), lr=para.learning_rate),
}

configs = {
      'repo': data,
      'model': model, 
      'criterion': Dice_BCE(smooth=1e-6, label_smoothing=None, class_weight=class_weight, device='cpu'),
      'metric': Dice_3D(binary=False),
      'fold': None, 
      'num_epoch': 200, 
      'optimizer': opt_configs['Adam'],
      'scheduler': optim.lr_scheduler.ReduceLROnPlateau(opt_configs['Adam'], mode='min', factor=0.5, patience=2),
      'verbose': False,
      'save_path':
            {'log': 'log_UnetRes3D_softmax',
             'best_model': 'best_UnetRes3D_softmax'},
      }


In [9]:
runner = Trainer(
                 repo = configs['repo'], 
                 model = configs['model'], 
                 criterion = configs['criterion'], 
                 optimizer = configs['optimizer'],
                 scheduler = configs['scheduler'],
                 metric = configs['metric'],
                 fold = configs['fold'], 
                 num_epochs = configs['num_epoch'],
                 verbose = configs['verbose'], 
                 path    = configs['save_path'],
                 configs =  None
                 )

cpu
fold None
verbose: False
data train: 4
img shape: torch.Size([1, 80, 160, 160])
seg shape: torch.Size([3, 80, 160, 160])
seg value: [0. 1.]


In [10]:
# runner.minibatch()


In [11]:
# runner.run(mode = ['train'])


 25%|██▌       | 1/4 [01:19<03:59, 79.79s/it]


KeyboardInterrupt: 